In [2]:
import pandas as pd
import requests

In [65]:
#note that api has 500 limit usage per hour or smth... look into that later down the line
#api from airnow, currently pulling from Illinois 

api_key = '4787639B-F250-4D11-B226-EF9E6579C8B0'
url = 'https://www.airnowapi.org/aq/data/?startDate=2025-05-19T00&endDate=2025-05-19T23&parameters=OZONE,PM25,NO2&BBOX=-91.600952,37.035168,-87.382202,42.599341&dataType=A&format=application/json&verbose=0&monitorType=0&includerawconcentrations=1&API_KEY=4787639B-F250-4D11-B226-EF9E6579C8B0'
response = requests.get(url)
list = response.json()

In [68]:
data = pd.DataFrame(list).reindex(columns = ['UTC', 'Latitude', 'Longitude', 'Parameter', 'RawConcentration', 'Unit', 'AQI', 'Category'])
data

,UTC,Latitude,Longitude,Parameter,RawConcentration,Unit,AQI,Category
0,2025-05-19T00:00,42.280600,-91.526900,OZONE,38.0,PPB,37,1
1,2025-05-19T00:00,41.658195,-91.503471,PM2.5,3.9,UG/M3,19,1
2,2025-05-19T00:00,39.917500,-91.335800,OZONE,43.0,PPB,40,1
3,2025-05-19T00:00,41.401400,-91.068400,PM2.5,5.0,UG/M3,24,1
4,2025-05-19T00:00,39.045857,-90.849189,OZONE,47.0,PPB,41,1
...,...,...,...,...,...,...,...,...
2712,2025-05-19T23:00,41.594427,-87.495044,PM2.5,13.9,UG/M3,56,2
2713,2025-05-19T23:00,38.740800,-87.485300,OZONE,52.0,PPB,48,1
2714,2025-05-19T23:00,41.653446,-87.435435,PM2.5,8.6,UG/M3,46,1
2715,2025-05-19T23:00,39.485987,-87.401312,OZONE,45.0,PPB,40,1


In [69]:
#The AQI category of the value. For more information refer to API Colors section of the Air Quality 101 page. A -999 Category reading indicates a missing or invalid AQI Category.
data_c = data[data['Category'] != -999]
data_c

,UTC,Latitude,Longitude,Parameter,RawConcentration,Unit,AQI,Category
0,2025-05-19T00:00,42.280600,-91.526900,OZONE,38.0,PPB,37,1
1,2025-05-19T00:00,41.658195,-91.503471,PM2.5,3.9,UG/M3,19,1
2,2025-05-19T00:00,39.917500,-91.335800,OZONE,43.0,PPB,40,1
3,2025-05-19T00:00,41.401400,-91.068400,PM2.5,5.0,UG/M3,24,1
4,2025-05-19T00:00,39.045857,-90.849189,OZONE,47.0,PPB,41,1
...,...,...,...,...,...,...,...,...
2712,2025-05-19T23:00,41.594427,-87.495044,PM2.5,13.9,UG/M3,56,2
2713,2025-05-19T23:00,38.740800,-87.485300,OZONE,52.0,PPB,48,1
2714,2025-05-19T23:00,41.653446,-87.435435,PM2.5,8.6,UG/M3,46,1
2715,2025-05-19T23:00,39.485987,-87.401312,OZONE,45.0,PPB,40,1


In [70]:
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
geolocator = Nominatim(user_agent='myGeocoder')
geocode = RateLimiter(geolocator.reverse, min_delay_seconds = 1)

def get_location(i):
    location = geocode((i['Latitude'], i['Longitude']), exactly_one = True)
    return location.address
data_c['Location'] = data.apply(get_location, axis = 1)

RateLimiter caught an error, retrying (0/2 tries). Called with (*((41.8131, -88.0728),), **{'exactly_one': True}).
Traceback (most recent call last):
  File "c:\Users\Lyons\OneDrive\Desktop\AQI-project\.venv\Lib\site-packages\urllib3\connection.py", line 198, in _new_conn
    sock = connection.create_connection(
        (self._dns_host, self.port),
    ...<2 lines>...
        socket_options=self.socket_options,
    )
  File "c:\Users\Lyons\OneDrive\Desktop\AQI-project\.venv\Lib\site-packages\urllib3\util\connection.py", line 85, in create_connection
    raise err
  File "c:\Users\Lyons\OneDrive\Desktop\AQI-project\.venv\Lib\site-packages\urllib3\util\connection.py", line 73, in create_connection
    sock.connect(sa)
    ~~~~~~~~~~~~^^^^
TimeoutError: timed out

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "c:\Users\Lyons\OneDrive\Desktop\AQI-project\.venv\Lib\site-packages\urllib3\connectionpool.py", line 787, in urlope

In [71]:
data_c

,UTC,Latitude,Longitude,Parameter,RawConcentration,Unit,AQI,Category,Location
0,2025-05-19T00:00,42.280600,-91.526900,OZONE,38.0,PPB,37,1,"506, Linn Street, Coggon, Linn County, Iowa, 5..."
1,2025-05-19T00:00,41.658195,-91.503471,PM2.5,3.9,UG/M3,19,1,"Iowa City High School, Heather Court, Iowa Cit..."
2,2025-05-19T00:00,39.917500,-91.335800,OZONE,43.0,PPB,40,1,"John Wood Community College, 1301, South 48th ..."
3,2025-05-19T00:00,41.401400,-91.068400,PM2.5,5.0,UG/M3,24,1,"Garfield School, Schley Avenue, Muscatine, Mus..."
4,2025-05-19T00:00,39.045857,-90.849189,OZONE,47.0,PPB,41,1,Winfield-Foley Fire Protection District Statio...
...,...,...,...,...,...,...,...,...,...
2712,2025-05-19T23:00,41.594427,-87.495044,PM2.5,13.9,UG/M3,56,2,"165th Street, Hammond, North Township, Lake Co..."
2713,2025-05-19T23:00,38.740800,-87.485300,OZONE,52.0,PPB,48,1,"Knox County, Indiana, United States"
2714,2025-05-19T23:00,41.653446,-87.435435,PM2.5,8.6,UG/M3,46,1,"Ameristar Casino Hotel East Chicago, 777, Aldi..."
2715,2025-05-19T23:00,39.485987,-87.401312,OZONE,45.0,PPB,40,1,"1111, Beech Street, Twelve Points, Terre Haute..."
